# Save Model To Inference Area

In [ ]:
inference_path = 'vertex_test/external_files/inference_model'

# Name of model you Fine Tuned on
pipe_name ="CompVis/stable-diffusion-v1-4"

# Replace this with whichever model you'd like to inference
import torch
from diffusers import StableDiffusionPipeline

# This will output the model structure to our inference area
pipe = StableDiffusionPipeline.from_pretrained(pipe_name, torch_dtype=torch.float16)
pipe.to('cuda')

pipe.save_pretrained(inference_path)

# Test TorchServe

In [9]:
!torch-model-archiver --model-name stable-diffusion --version 1.0 --handler vertex_test/stable_diffusion_handler.py --export-path vertex_test/model-store -f --extra-files vertex_test/external_files

WARNING - Overwriting vertex_test/model-store/stable-diffusion.mar ...


In [7]:
!torchserve --start  --models=stable-diffusion.mar --model-store=vertex_test/model-store --ts-config=config.properties

# Test Docker

In [ ]:
!docker build -t serve_sd .

In [ ]:
!docker run --runtime=nvidia -p 7080:7080 serve_sd 

# Test Response Locally

In [10]:
import requests
prompt = "A crazy dog running around"
URL = "http://127.0.0.1:7080/predictions/stable-diffusion"
response = requests.post(URL, data=prompt)

# Test On Vertex AI After Deploy

In [ ]:
from google.cloud import aiplatform as aip

PROJECT_NAME = "XXXXXXXXXXXXXXXXXX" # Replace with your project name
REGION="us-central1" # Replace with the region you've deployed to
ENDPOINT_ID = "6166813274521206784" # Replace with your endpoint
 
aip.init(project=PROJECT_NAME, location=REGION)
endpoint = aip.Endpoint(endpoint_name=ENDPOINT_ID)
text_input = """A crazy dog running around"""

def query_endpoint(endpoint, text_input):
 payload = {"data": text_input}
 response = endpoint.predict(instances=[payload])
 return response

image = query_endpoint(endpoint, text_input)

# Remove Model and Endpoint From Vertex AI

In [ ]:
endpoints = aip.Endpoint.list()
for i in endpoints:
        i.undeploy_all()